In [ ]:
'''
Project: Bangkit Capstone Project
Group: B21-CAP0194
Part: Modelling
'''

'\nProject: Bangkit Capstone Project\nGroup: B21-CAP0194\nPart: Modelling\n'

In [124]:
vocab_size = 20000
embedding_dim = 32
max_length = 32
trunc_type = 'post'
oov_tok = "<OOV>"

In [106]:
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [107]:
#Import Labelled Dataset
url = 'https://drive.google.com/file/d/1nOn8OONtZA7pcX4Pvh8W_wOUg49ZWTvT/view?usp=sharing'
file_id = url.split('/')[-2]
csv_url = 'https://drive.google.com/uc?id=' + file_id
df = pd.read_csv(csv_url)
df.head()

,index,text,label
0,0,the ironies of the movement are too painfully...,private places
1,1,women in kuwait have launched a social media c...,public places
2,2,triger warning raising awareness of sexual ass...,private places
3,3,charges brought against the guy that smacked m...,private places
4,4,and the fourth presenter on 26th may is hannah...,educational places


In [ ]:
df_cut = df[:100]

In [ ]:
df_cut.head(50)

,index,text,label
0,0,the ironies of the movement are too painfully...,work places
1,1,women in kuwait have launched a social media c...,cyber
2,2,triger warning raising awareness of sexual ass...,private places
3,3,charges brought against the guy that smacked m...,private places
4,4,and the fourth presenter on 26th may is hannah...,educational places
5,5,this is so messed up are there any men in the...,public places
6,6,to tackle an epidemic of male violence against...,cyber
7,7,there has been a new series of harassment alle...,private places
8,8,want to see what the movement looks like its ...,public places
9,9,female students dont want to have sex w dolan ...,educational places


In [ ]:
import getpass

APIKEY = getpass.getpass()

In [ ]:
# running Translate API
#from googleapiclient.discovery import build
#service = build('translate', 'v2', developerKey=APIKEY)

#df_cut['indo_text'] = df_cut['text'].apply(lambda x: (service.translations().list(source='en', target='id', q=x).execute())).apply(lambda x: x['translations']).apply(lambda x: x[0]).apply(lambda x: x['translatedText'])

In [ ]:
# running Translate API
from googleapiclient.discovery import build
service = build('translate', 'v2', developerKey=APIKEY)

df['indo_text'] = df['text'].apply(lambda x: (service.translations().list(source='en', target='id', q=x).execute())).apply(lambda x: x['translations']).apply(lambda x: x[0]).apply(lambda x: x['translatedText'])

In [ ]:
df['indo_text'].head(50)

In [ ]:
#Save DataFrame to CSV
df_translated = pd.DataFrame()
df_translated['index'] = df['index']
df_translated['text'] = df['text']
df_translated['indo_text'] = df['indo_text']
df_translated.to_csv("capstonetweettranslatednew.csv", index=False)

In [108]:
labels = pd.get_dummies(df.label)

In [109]:
df_new = pd.concat([df, labels], axis=1)
df_new = df_new.drop(columns='label')

In [110]:
tweet = df_new['text'].values
label = df_new[["cyber", "educational places", "neutral", "private places", "public places", "work places"]].values

In [111]:
label

array([[0, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 1, 0],
       [0, 0, 0, 1, 0, 0],
       ...,
       [0, 0, 0, 0, 1, 0],
       [0, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 1, 0]], dtype=uint8)

In [112]:
from sklearn.model_selection import train_test_split
#Train Text, Test Text, Train Label, Test Label
X_train, X_test, y_train, y_test = train_test_split(tweet, label, test_size=0.2, shuffle=True)

In [113]:
tokenizer = Tokenizer(num_words = vocab_size, oov_token = oov_tok)
tokenizer.fit_on_texts(X_train)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(X_train)
padded = pad_sequences(sequences, maxlen=max_length, truncating=trunc_type)

testing_sequences = tokenizer.texts_to_sequences(X_test)
testing_padded = pad_sequences(testing_sequences, maxlen = max_length)

In [125]:
class myCallback(tf.keras.callbacks.Callback):
	def on_epoch_end(self, epoch, logs={}):
		if(logs.get('val_accuracy')>0.90):
			print("Desired accuracy reached!")
			self.model.stop_training=True

In [127]:
import tensorflow as tf
'''
model = tf.keras.Sequential([
  tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
  #tf.keras.layers.Flatten(),
  tf.keras.layers.LSTM(64),
  tf.keras.layers.Dense(32, activation='relu'),
  tf.keras.layers.Dense(6, activation='softmax')
])

'''
'''
model = tf.keras.Sequential([
  tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
  #tf.keras.layers.Flatten(),
  tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True)),
  tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
  tf.keras.layers.Dense(32, activation='relu'),
  tf.keras.layers.Dense(6, activation='softmax')
])
'''
model = tf.keras.Sequential([
  tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
  #tf.keras.layers.Flatten(),
  tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128, return_sequences=True)),
  tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True)),
  tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32, return_sequences=True)),
  tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(16)),
  tf.keras.layers.Dropout(0.1),
  tf.keras.layers.Dense(32, activation='relu'),
  tf.keras.layers.Dense(6, activation='softmax')
])


transfer learning models for NLP: BERT, coba buka ini https://www.dlology.com/blog/keras-meets-universal-sentence-encoder-transfer-learning-for-text-data/

In [128]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_10 (Embedding)     (None, 32, 32)            640000    
_________________________________________________________________
bidirectional_25 (Bidirectio (None, 32, 256)           164864    
_________________________________________________________________
bidirectional_26 (Bidirectio (None, 32, 128)           164352    
_________________________________________________________________
bidirectional_27 (Bidirectio (None, 32, 64)            41216     
_________________________________________________________________
bidirectional_28 (Bidirectio (None, 32)                10368     
_________________________________________________________________
dropout_1 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_18 (Dense)             (None, 32)               

In [129]:
num_epochs = 100
callbacks = myCallback()
model.fit(padded, y_train, epochs=num_epochs, validation_data=(testing_padded, y_test), callbacks=[callbacks])

Epoch 1/100
353/353 [==============================] - 21s 33ms/step - loss: 1.3286 - accuracy: 0.4278 - val_loss: 0.8722 - val_accuracy: 0.6942
Epoch 2/100
353/353 [==============================] - 9s 27ms/step - loss: 0.6728 - accuracy: 0.7764 - val_loss: 0.6052 - val_accuracy: 0.8005
Epoch 3/100
353/353 [==============================] - 9s 27ms/step - loss: 0.3950 - accuracy: 0.8630 - val_loss: 0.5990 - val_accuracy: 0.8101
Epoch 4/100
353/353 [==============================] - 9s 27ms/step - loss: 0.2498 - accuracy: 0.9145 - val_loss: 0.6109 - val_accuracy: 0.8342
Epoch 5/100
353/353 [==============================] - 9s 26ms/step - loss: 0.1677 - accuracy: 0.9377 - val_loss: 0.6364 - val_accuracy: 0.8469
Epoch 6/100
353/353 [==============================] - 9s 27ms/step - loss: 0.1426 - accuracy: 0.9506 - val_loss: 0.7309 - val_accuracy: 0.8441
Epoch 7/100
353/353 [==============================] - 9s 27ms/step - loss: 0.1140 - accuracy: 0.9633 - val_loss: 0.7233 - val_accuracy

In [130]:
text = ["i meet my mom and my dad at home"]

tokenized_text = tokenizer.texts_to_sequences(text)

In [131]:
prediction = model.predict(tokenized_text)

In [132]:
labels = ["cyber", "educational places", "neutral", "private places", "public places", "work places"]
predicted = prediction.argmax(axis=1)
predict_labels = [labels[predict] for predict in predicted]
print(prediction)
print(predict_labels)

[[1.5231024e-12 5.3711265e-08 3.9630859e-09 1.0000000e+00 5.4070203e-08
  6.2733052e-12]]
['private places']


In [133]:
#Exporting the Model (TFLITE)
export_dir = 'saved_model/1'
tf.saved_model.save(model, export_dir)

INFO:tensorflow:Assets written to: saved_model/1/assets


INFO:tensorflow:Assets written to: saved_model/1/assets


In [134]:
converter = tf.lite.TFLiteConverter.from_saved_model(export_dir)
#converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

In [135]:
import pathlib

tflite_model_file = pathlib.Path('/content/model.tflite')
tflite_model_file.write_bytes(tflite_model)

4156832

In [136]:
from google.colab import files
files.download(tflite_model_file)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [139]:
#Exporting the Tokenized Word
import json

with open( 'word_dict.json' , 'w' ) as file:
    json.dump( tokenizer.word_index , file )